In [1]:
import math
from collections import defaultdict

class MovieRecommender:
    """
    Um sistema simples de recomendação de filmes baseado em
    similaridade de cosseno entre usuários (User-Based Collaborative Filtering).
    """

    def __init__(self, user_movie_data):
        """
        Inicializa o sistema de recomendação.

        Args:
            user_movie_data (dict): Um dicionário onde as chaves são user_ids
                                     e os valores são sets de movie_ids que o
                                     usuário assistiu.
                                     Ex: {'user1': {'movieA', 'movieB'},
                                          'user2': {'movieB', 'movieC'}}
        """
        self.user_movie_data = user_movie_data
        # Cria um set com todos os filmes únicos para facilitar cálculos
        self.all_movies = set()
        for movies in user_movie_data.values():
            self.all_movies.update(movies)
        print(f"Sistema inicializado com {len(user_movie_data)} usuários e {len(self.all_movies)} filmes.")

    def _calculate_cosine_similarity(self, user1_id, user2_id):
        """Calcula a similaridade de cosseno entre dois usuários."""
        movies_user1 = self.user_movie_data.get(user1_id, set())
        movies_user2 = self.user_movie_data.get(user2_id, set())

        # Encontra filmes assistidos por ambos
        common_movies = movies_user1.intersection(movies_user2)

        # Se não há filmes em comum, a similaridade é 0
        if not common_movies:
            return 0.0

        # Produto escalar (como estamos usando sets - assistiu/não assistiu -
        # o produto escalar é simplesmente o número de filmes em comum)
        dot_product = len(common_movies)

        # Calcula as magnitudes (normas) dos vetores de cada usuário
        # Magnitude = raiz quadrada do número de filmes assistidos
        magnitude_user1 = math.sqrt(len(movies_user1))
        magnitude_user2 = math.sqrt(len(movies_user2))

        # Evita divisão por zero se um usuário não assistiu nada (embora improvável com sets)
        if magnitude_user1 == 0 or magnitude_user2 == 0:
            return 0.0

        # Calcula a similaridade de cosseno
        similarity = dot_product / (magnitude_user1 * magnitude_user2)
        return similarity

    def find_similar_users(self, target_user_id, n_similar=5):
        """Encontra os N usuários mais similares ao usuário alvo."""
        if target_user_id not in self.user_movie_data:
            print(f"Erro: Usuário {target_user_id} não encontrado na base de dados.")
            return []

        similarities = []
        for other_user_id in self.user_movie_data:
            if other_user_id != target_user_id:
                similarity = self._calculate_cosine_similarity(target_user_id, other_user_id)
                if similarity > 0: # Considera apenas usuários com alguma similaridade
                    similarities.append((other_user_id, similarity))

        # Ordena os usuários pela similaridade em ordem decrescente
        similarities.sort(key=lambda item: item[1], reverse=True)

        return similarities[:n_similar]

    def recommend_movies(self, target_user_id, n_recommendations=5):
        """Recomenda filmes para um usuário alvo baseado nos usuários mais similares."""
        if target_user_id not in self.user_movie_data:
            print(f"Erro: Usuário {target_user_id} não encontrado.")
            return []

        similar_users = self.find_similar_users(target_user_id)

        if not similar_users:
            print(f"Não foram encontrados usuários similares para {target_user_id}.")
            return []

        print(f"\nUsuários mais similares a {target_user_id}:")
        for user, sim in similar_users:
            print(f"- {user} (Similaridade: {sim:.4f})")

        # Agrega pontuações para filmes assistidos por usuários similares
        recommendation_scores = defaultdict(float)
        target_watched_movies = self.user_movie_data[target_user_id]

        for similar_user_id, similarity_score in similar_users:
            similar_user_movies = self.user_movie_data[similar_user_id]
            for movie_id in similar_user_movies:
                # Recomenda apenas filmes que o usuário alvo NÃO assistiu
                if movie_id not in target_watched_movies:
                    # A pontuação do filme é a soma das similaridades dos usuários
                    # que o assistiram
                    recommendation_scores[movie_id] += similarity_score

        # Ordena as recomendações pela pontuação calculada
        sorted_recommendations = sorted(recommendation_scores.items(),
                                        key=lambda item: item[1],
                                        reverse=True)

        return [movie for movie, score in sorted_recommendations[:n_recommendations]]

# --- Exemplo de Uso ---

# 1. Modelo (Dados de exemplo: quais filmes cada usuário assistiu)
#    Usaremos sets para indicar os filmes assistidos.
user_movie_data = {
    'Alice': {'Matrix', 'Interestelar', 'Blade Runner 2049', 'O Senhor dos Anéis'},
    'Bob': {'Matrix', 'Interestelar', 'Vingadores Ultimato', 'Star Wars V'},
    'Charlie': {'O Poderoso Chefão', 'Clube da Luta', 'Forrest Gump'},
    'David': {'Matrix', 'Blade Runner 2049', 'Interestelar', 'A Origem'},
    'Eve': {'Vingadores Ultimato', 'Star Wars V', 'Guardiões da Galáxia', 'Matrix'},
    'Frank': {'O Senhor dos Anéis', 'Harry Potter 1', 'O Hobbit'},
    'Grace': {'Interestelar', 'A Origem', 'Gravidade', 'Perdido em Marte'}
}

# 2. Criação da instância do sistema
recommender = MovieRecommender(user_movie_data)

# 3. Geração de Recomendações (Previsões)
target_user = 'Alice'
print(f"\nFilmes assistidos por {target_user}: {recommender.user_movie_data.get(target_user, set())}")

recommendations = recommender.recommend_movies(target_user, n_recommendations=3)

print(f"\nRecomendações para {target_user}:")
if recommendations:
    for i, movie in enumerate(recommendations):
        print(f"{i+1}. {movie}")
else:
    print("Nenhuma recomendação encontrada.")

# Exemplo para outro usuário
target_user_2 = 'Bob'
print(f"\nFilmes assistidos por {target_user_2}: {recommender.user_movie_data.get(target_user_2, set())}")
recommendations_2 = recommender.recommend_movies(target_user_2, n_recommendations=3)
print(f"\nRecomendações para {target_user_2}:")
if recommendations_2:
    for i, movie in enumerate(recommendations_2):
        print(f"{i+1}. {movie}")
else:
    print("Nenhuma recomendação encontrada.")

Sistema inicializado com 7 usuários e 15 filmes.

Filmes assistidos por Alice: {'Matrix', 'Interestelar', 'O Senhor dos Anéis', 'Blade Runner 2049'}

Usuários mais similares a Alice:
- David (Similaridade: 0.7500)
- Bob (Similaridade: 0.5000)
- Frank (Similaridade: 0.2887)
- Eve (Similaridade: 0.2500)
- Grace (Similaridade: 0.2500)

Recomendações para Alice:
1. A Origem
2. Vingadores Ultimato
3. Star Wars V

Filmes assistidos por Bob: {'Matrix', 'Interestelar', 'Vingadores Ultimato', 'Star Wars V'}

Usuários mais similares a Bob:
- Eve (Similaridade: 0.7500)
- Alice (Similaridade: 0.5000)
- David (Similaridade: 0.5000)
- Grace (Similaridade: 0.2500)

Recomendações para Bob:
1. Blade Runner 2049
2. Guardiões da Galáxia
3. A Origem
